# Imports
Import required libraries for the VRP solver, including JSON handling, itertools for permutations, math for infinity, time for performance measurement, NetworkX for graph operations, and Matplotlib for visualization.

In [ ]:
import json
import itertools
import math
import time
import networkx as nx
import matplotlib.pyplot as plt

# VRP Brute Force Solver Class
Define the VRPBruteForceSolver class, which handles loading the cost matrix, calculating distances, generating group sizes, and solving the Vehicle Routing Problem using a brute-force approach.

In [ ]:
class VRPBruteForceSolver:
    def __init__(self, cost_matrix_path, capacity=3):
        self.cost_matrix_path = cost_matrix_path
        self.capacity = capacity
        self.cost_matrix = None
        self.patients = []
        self.hospital = "Hospital"
        
    def load_cost_matrix(self):
        """Load cost matrix from JSON file"""
        with open(self.cost_matrix_path, "r") as f:
            self.cost_matrix = json.load(f)
        self.patients = [node for node in self.cost_matrix.keys() if node != self.hospital]
    
    def get_distance(self, location_a, location_b):
        """Get distance between two locations from cost matrix"""
        return float(self.cost_matrix[location_a][location_b])
    
    def calculate_trip_cost(self, trip):
        """Calculate total cost for a single trip including return to hospital"""
        current_location = self.hospital
        total_cost = 0.0
        
        for patient in trip:
            total_cost += self.get_distance(current_location, patient)
            current_location = patient
            
        # Add return trip to hospital
        total_cost += self.get_distance(current_location, self.hospital)
        return total_cost
    
    def generate_group_sizes(self, num_patients, max_group_size):
        """
        Generate all possible ways to split patients into groups of specified maximum size
        
        Args:
            num_patients: Total number of patients to split
            max_group_size: Maximum number of patients per group
            
        Returns:
            List of lists representing different grouping options
        """
        valid_splits = []
        
        def _generate_splits(remaining_patients, current_split):
            if remaining_patients == 0:
                valid_splits.append(current_split.copy())
                return
            
            for group_size in range(1, min(max_group_size, remaining_patients) + 1):
                current_split.append(group_size)
                _generate_splits(remaining_patients - group_size, current_split)
                current_split.pop()
                
        _generate_splits(num_patients, [])
        return valid_splits
    
    def solve(self):
        """Brute force solution to find optimal VRP solution"""
        num_patients = len(self.patients)
        best_cost = math.inf
        best_plan = None
        
        # Generate all possible ways to split patients into groups
        group_size_options = self.generate_group_sizes(num_patients, self.capacity)
        
        start_time = time.time()
        
        # Try all permutations of patients and all possible groupings
        for patient_permutation in itertools.permutations(self.patients):
            for group_sizes in group_size_options:
                current_plan = []
                patient_index = 0
                
                # Create trips based on group sizes
                for group_size in group_sizes:
                    trip = tuple(patient_permutation[patient_index:patient_index + group_size])
                    current_plan.append(trip)
                    patient_index += group_size
                
                # Calculate total cost for this plan
                total_cost = sum(self.calculate_trip_cost(trip) for trip in current_plan)
                
                # Update best solution if better
                if total_cost < best_cost:
                    best_cost = total_cost
                    best_plan = current_plan
                    
        elapsed_time = time.time() - start_time
        return best_plan, best_cost, elapsed_time

# Visualization Function
Define the visualization function to display the optimal VRP solution using NetworkX and Matplotlib, showing nodes, edges, and distances.

In [ ]:
def visualize_solution(plan, total_cost, hospital="Hospital", cost_matrix=None):
    """
    Visualize the VRP solution using networkx
    
    Args:
        plan: List of trips (each trip is a tuple of patient names)
        total_cost: Total cost of the solution
        hospital: Name of the hospital location
        cost_matrix: Cost matrix for distances between locations
    """
    graph = nx.Graph()
    
    # Add nodes to graph
    graph.add_node(hospital)
    for trip in plan:
        for patient in trip:
            graph.add_node(patient)
    
    # Add edges to graph
    for trip in plan:
        current_location = hospital
        for patient in trip:
            graph.add_edge(current_location, patient, weight=cost_matrix[current_location][patient])
            current_location = patient
        graph.add_edge(current_location, hospital, weight=cost_matrix[current_location][hospital])
    
    # Position nodes using spring layout
    positions = nx.spring_layout(graph, seed=42)
    
    # Color nodes differently for hospital and patients
    node_colors = ["lightblue" if node == hospital else "lightgreen" for node in graph.nodes()]
    
    # Create visualization
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(graph, positions, node_size=800, node_color=node_colors, edgecolors="black")
    nx.draw_networkx_labels(graph, positions, font_size=10, font_weight="bold")
    nx.draw_networkx_edges(graph, positions, width=2, edge_color="gray")
    
    # Add edge labels with weights
    edge_labels = nx.get_edge_attributes(graph, 'weight')
    formatted_edge_labels = {k: f"{v:.1f}" for k, v in edge_labels.items()}
    nx.draw_networkx_edge_labels(graph, positions, edge_labels=formatted_edge_labels)
    
    plt.title(f"Optimal VRP Solution\nTotal Cost = {total_cost:.2f}")
    plt.axis("off")
    plt.show()

# Main Execution
Run the VRP solver with a sample cost matrix and visualize the results. Note: Update the cost_matrix_path to a valid JSON file path in your Colab environment.

In [ ]:
# Main execution
if __name__ == "__main__":
    # Initialize and run VRP solver
    vrp_solver = VRPBruteForceSolver("cost_matrix.json", capacity=3)
    vrp_solver.load_cost_matrix()
    
    optimal_plan, optimal_cost, computation_time = vrp_solver.solve()
    
    print(f"Brute-force Solution: cost = {optimal_cost:.4f}, time = {computation_time:.4f}s, plan = {optimal_plan}")
    
    # Visualize the solution
    visualize_solution(optimal_plan, optimal_cost, vrp_solver.hospital, vrp_solver.cost_matrix)

# Notes
- Ensure the `cost_matrix.json` file is uploaded to your Colab environment before running the main execution cell.
- The JSON file should contain a dictionary where keys are location names (including 'Hospital') and values are dictionaries mapping to other locations with their respective distances.
- Example JSON format:
```json
{
    "Hospital": {"Patient1": 10.0, "Patient2": 15.0, ...},
    "Patient1": {"Hospital": 10.0, "Patient2": 5.0, ...},
    ...
}
```